<a href="https://colab.research.google.com/github/mayank-cse/DEV---A-Face-Identification-Software/blob/main/webPageCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
! pip install streamlit -q
! pip install pyngrok
! pip install --upgrade protobuf
!pip install facenet-pytorch
!pip install -v -e
# !pip install mmcv-full
!pip install -U openmim
!pip install mmcv
!pip install scikit-image
!pip install -q streamlit
!pip install pyngrok

     |████████████████████████████████| 1.0 MB 5.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-4.21.12-cp37-abi3-manylinux2014_x86_64.whl (409 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [66]:
%%writefile webapp.py

import streamlit as st
import cv2
from PIL import Image
import numpy as np
# importing libraries

from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import skimage.io
import io

# from scipy.misc import impip save
from imageio import imwrite
from google.colab.patches import cv2_imshow

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from skimage import data, img_as_float
from skimage import exposure
from PIL import Image, ImageEnhance

from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display
    
# loading data.pt file
load_data = torch.load('data.pt') 
embedding_list = load_data[0] 
name_list = load_data[1] 

# initializing MTCNN and InceptionResnetV1 
mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all=False
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) # keep_all=True
# mtcnn = MTCNN(keep_all=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval()
def improveQuality(img):
    img = np.array(img.convert('RGB'))
    #Enhance Image Pixels
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    path = "/content/EDSR_x4.pb"
    sr.readModel(path)
    sr.setModel("edsr",4)
    result = sr.upsample(img)
    return result
def enhance_faces(img):
    # Contrast stretching
    img = np.array(img.convert('RGB'))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_adapteq = exposure.equalize_adapthist(gray, clip_limit=0.03)
    #img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img_adapteq
def detect_faces(frame):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    img = np.array(frame.convert('RGB'))
    mtcnn = MTCNN(keep_all=True, device=device)
    img = np.array(frame.convert('RGB'))
    #img = Image.fromarray(frame)
    img_cropped_list, prob_list = mtcnn(img, return_prob=True) 
    #print(prob_list)
    boxes, _ = mtcnn.detect(img)
    #print(boxes)
    if img_cropped_list is not None:
        boxes, _ = mtcnn.detect(img)
        #print(prob_list)        
        for i, prob in enumerate(prob_list):
            if prob>0.80:
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 
                
                dist_list = [] # list of matched distances, minimum distance is used to identify the person
                
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list) # get minumum dist value
                min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                name = name_list[min_dist_idx] # get name corrosponding to minimum dist
                
                box = boxes[i] 
                
                original_frame = frame.copy() # storing copy of frame before drawing on it
                
                
                frame = cv2.rectangle(img, (int(box[0]),int(box[1])) , (int(box[2]),int(box[3])), (255,0,0), 2)
    return img
def recog_faces(frame):
    img = np.array(frame.convert('RGB'))
    #img = Image.fromarray(frame)
    img_cropped_list, prob_list = mtcnn(img, return_prob=True) 
    print(prob_list)
    boxes, _ = mtcnn.detect(img)
    #print(boxes)
    if img_cropped_list is not None:
        boxes, _ = mtcnn.detect(img)
        #print(prob_list)        
        for i, prob in enumerate(prob_list):
            if prob>0.80:
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 
                
                dist_list = [] # list of matched distances, minimum distance is used to identify the person
                
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list) # get minumum dist value
                min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                name = name_list[min_dist_idx] # get name corrosponding to minimum dist
                
                box = boxes[i] 
                
                original_frame = frame.copy() # storing copy of frame before drawing on it
                
                if min_dist<0.85:
                    print(box[0],box[1])
                    frame = cv2.putText(img, name+' '+str(min_dist), (int(box[0]), int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA, False)
                    print((box[0],box[1]) , (box[2],box[3]))
                frame = cv2.rectangle(img, (int(box[0]),int(box[1])) , (int(box[2]),int(box[3])), (255,0,0), 2)
    return img
def main():
    """Face Recognition App"""

    st.title("Mayank Streamlit Tutorial")

    html_temp = """
    <body style="background-color:red;background-image ='https://drive.google.com/file/d/12BUgTqnvSoNl2wOLudNFKdkNcvaD2UvG/view?usp=share_link'">
    <div style="background-color:teal; padding:10px;">
    <h2 style="color:white;text-align:center;">Face Recognition WebApp</h2>
    </div>
    </body>
    """

    st.markdown(html_temp, unsafe_allow_html=True)

    image_file = st.file_uploader("Upload Image", type=['jpg', 'png', 'jpeg'])
    if image_file is not None:
        our_image = Image.open(image_file)
        st.text("Original Image")
        st.image(our_image)

    if st.button("Recognise"):
        result_img= recog_faces(our_image)
        st.image(result_img)

    if st.button("ImageEnhance"):
        result_img= enhance_faces(our_image)
        st.image(result_img)

    if st.button("Image Quality Improve"):
        result_img = improveQuality(our_image)
        st.image(result_img)

    if st.button("Face Detection"):
        result_img = detect_faces(our_image)
        st.image(result_img)
if __name__ == '__main__':
    main()

Overwriting webapp.py


In [8]:
from pyngrok import ngrok

In [9]:
ngrok.set_auth_token("2JOk1LlPWzFvC9AWEkfebDsaorh_7uqdTpaKTJCUMQju78Muc") #ngrok.com

In [69]:
# correct
!streamlit run webapp.py --server.port 80 &
url = ngrok.connect(port='80')
print(url)




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:80
  External URL: http://34.123.226.134:80

2022-12-25 16:39:44.105 Generating grammar tables from /usr/lib/python3.8/lib2to3/Grammar.txt
2022-12-25 16:39:44.127 Generating grammar tables from /usr/lib/python3.8/lib2to3/PatternGrammar.txt
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
  Stopping...
NgrokTunnel: "http://aacf-34-123-226-134.ngrok.io" -> "http://localhost:80"
